In [2]:
#import package
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

In [5]:
#load data
data=pd.read_csv("train.csv")

X=data.iloc[:,0:-1]
y=data.iloc[:,-1]

train_X,test_X,train_y,test_y=train_test_split(X,y,train_size=0.8,random_state=3)
#data preprocessing
#標準化
sc=StandardScaler()
ad_train_X=train_X.iloc[:,[3,6,7,8,9,12]]
ad_test_X=test_X.iloc[:,[3,6,7,8,9,12]]
ad1_train_X=sc.fit_transform(ad_train_X)
ad1_test_X=sc.transform(ad_test_X)
#重新編碼
ohe=OneHotEncoder()
ad2_train_X=train_X.iloc[:,[4,5,10,11]]
ad2_test_X=test_X.iloc[:,[4,5,10,11]]
ad3_train_X=ohe.fit_transform(ad2_train_X).toarray()
ad3_test_X=ohe.transform(ad2_test_X).toarray()

df1=pd.DataFrame(ad1_train_X)
df2=pd.DataFrame(ad3_train_X)
for i in range(6,15):
    df1[i]=df2[i-6]
df3=pd.DataFrame(ad1_test_X)
df4=pd.DataFrame(ad3_test_X)
for i in range(6,15):
    df3[i]=df4[i-6]

#去掉不必要的欄位
df1.drop(columns=[8,9,11,12,13],inplace=True)
df3.drop(columns=[8,9,11,12,13],inplace=True)

adj_train_X=df1
adj_test_X=df3

#指定模型
clf=XGBClassifier(eval_metric=['logloss','auc','error'],use_label_encoder=False)
#給定指定的參數範圍
param_dist = {
            'max_depth':[3],
            'learning_rate':np.linspace(0.1,0.35,50)
            }
#利用GridSearchCV找到最佳的參數組合
grid = GridSearchCV(clf,param_dist,cv = 3)
grid.fit(adj_train_X,train_y)
best_estimator = grid.best_estimator_
print(best_estimator)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric=['logloss', 'auc', 'error'], gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.20714285714285713, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)
